## Подготовка
1. Клонировать репозиторий
2. Скачать и сохранить ключ доступа к Google service account (json) в папку проекта. [Authenticating as a service account  |  Authentication  |  Google Cloud](https://cloud.google.com/docs/authentication/production)
3. В файле  `.env`  заполнить поля: 
	
	* **GOOGLE_APPLICATION_CREDENTIALS** — путь к 	Google service account , сохраненному на **шаге 3**. К примеру, `./prod_credentials.json`

	* **GOOGLE_BIGQUERY_PROJECT_ID** — название проекта в BigQuery _(где находятся сгенерированные таблицы с помощью ETL)_. К примеру, `ml-development-294708`

	* **GOOGLE_BIGQUERY_DEBUG_DATASET_ID** — название датасета  _(где находятся сгенерированные таблицы с помощью ETL)_. К примеру,  `WW_WIRE_PARDOT`
	
	* **GPU2** — доступна ли видеокарта для обучения

4. Установить необходимые для работы программы модули командой в терминале: 
`pip3 install numpy pandas-gbq envparse gdelt`



In [1]:
import pandas as pd

from pathlib import Path

In [2]:
from envparse import env

env.read_envfile()

In [19]:
issues = {"taxes" : ["ECON_TAXATION",],
"unemployment" : [ "UNEMPLOYMENT", ],
"domesticeconomy" : ["ECON_BANKRUPTCY", "ECON_BOYCOTT", "ECON_COST_OF_LIVING", "ECON_CUTOUTLOOK", "ECON_DEREGULATION", "ECON_EARNINGSREPORT", "ECON_ENTREPRENEURSHIP", "ECON_HOUSING_PRICES", "ECON_INFORMAL_ECONOMY", "ECON_IPO", "ECON_INTEREST_RATE", "ECON_MONOPOLY", "ECON_MOU", "ECON_NATIONALIZE", "ECON_PRICECONTROL", "ECON_REMITTANCE", "ECON_STOCKMARKET", "ECON_SUBSIDIES", "ECON_UNIONS", "SLFID_ECONOMIC_DEVELOPMENT", "SLFID_ECONOMIC_POWER", "SOC_ECONCOOP"],
"trade" : ["ECON_TRADE_DISPUTE", "ECON_FOREIGNINVEST", "ECON_FREETRADE", "ECON_CURRENCY_EXCHANGE_RATE", "ECON_CURRENCY_RESERVES", "ECON_DEBT"],
"terrorism" : ["TAX_TERROR_GROUP", "SUICIDE_ATTACK", "EXTREMISM", "JIHAD", "TERROR", "WMD"],
"military" : ["ACT_FORCEPOSTURE", "ARMEDCONFLICT", "BLOCKADE", "CEASEFIRE", "MILITARY", "MILITARY_COOPERATION", "PEACEKEEPING", "RELEASE_HOSTAGE", "SEIGE", "SLFID_MILITARY_BUILDUP", "SLFID_MILITARY_READINESS", "SLFID_MILITARY_SPENDING", "SLFID_PEACE_BUILDING", "TAX_MILITARY_TITLE"],
"internationalrelations" : ["GOV_INTERGOVERNMENTAL", "SOC_DIPLOMCOOP", "RELATIONS"],
"immigration/refugees" : ["BORDER", "CHECKPOINT", "DISPLACED",  "EXILE", "IMMIGRATION", "REFUGEES", "SOC_FORCEDRELOCATION", "SOC_MASSMIGRATION", "UNREST_CHECKPOINT", "UNREST_CLOSINGBORDER"],
"healthcare" : ["GENERAL_HEALTH", "HEALTH_SEXTRANSDISEASE", "HEALTH_VACCINATION", "MEDICAL", "MEDICAL_SECURITY"],
"guncontrol" : ["FIREARM_OWNERSHIP", "MIL_SELF_IDENTIFIED_ARMS_DEAL", "MIL_WEAPONS_PROLIFERATION"],
"drug" : ["CRIME_ILLEGAL_DRUGS", "DRUG_TRADE", "TAX_CARTELS", "CRIME_CARTELS"],
"policesystem" : ["UNREST_POLICEBRUTALITY", "SECURITY_SERVICES"],
"racism" : ["DISCRIMINATION", "HATE_SPEECH"],
"civilliberties" : ["GENDER_VIOLENCE", "LGBT", "MOVEMENT_SOCIAL",  "MOVEMENT_WOMENS", "SLFID_CIVIL_LIBERTIES"],
"environment" :  ["ENV_BIOFUEL", "ENV_CARBONCAPTURE", "ENV_CLIMATECHANGE",  "ENV_COAL", "ENV_DEFORESTATION", "ENV_FISHERY", "ENV_FORESTRY", "ENV_GEOTHERMAL", "ENV_GREEN", "ENV_HYDRO", "ENV_METALS", "ENV_MINING", "ENV_NATURALGAS", "ENV_NUCLEARPOWER", "ENV_OIL", "ENV_OVERFISH", "ENV_POACHING", "ENV_WATERWAYS ", "ENV_SOLAR", "ENV_SPECIESENDANGERED", "ENV_SPECIESEXTINCT", "ENV_WINDPOWER", "FUELPRICES", "MOVEMENT_ENVIRONMENTAL", "SELF_IDENTIFIED_ENVIRON_DISASTER", "SLFID_MINERAL_RESOURCES", "SLFID_NATURAL_RESOURCES", "WATER_SECURITY"],
"partypolitics" : ["TAX_POLITICAL_PARTY"],
"electionfraud" : ["ELECTION_FRAUD"],
"education" : ["EDUCATION"],
"media/internet" : ["CYBER_ATTACK",  "INTERNET_BLACKOUT", "INTERNET_CENSORSHIP", "MEDIA_CENSORSHIP", "MEDIA_MSM", "MEDIA_SOCIAL", "SURVEILLANCE", "FREESPEECH"],
}

In [20]:
', '.join(issues['internationalrelations'])

'GOV_INTERGOVERNMENTAL, SOC_DIPLOMCOOP, RELATIONS'

In [25]:
q = f"""
    SELECT SourceCommonName, theme, count from (
        SELECT SourceCommonName, theme, COUNT(*) as count
        FROM (
            SELECT 
                SourceCommonName, 
                SPLIT(V2Themes, ';') theme
            FROM `gdelt-bq.gdeltv2.gkg` 
            WHERE 
                DATE >= 20150200000000 AND 
                DATE < 20151099999999
        )
        CROSS JOIN UNNEST(theme) AS theme
        GROUP BY SourceCommonName, theme
    ) WHERE count > 100
    ORDER BY count DESC
"""
gbq_df = pd.read_gbq(
        q, # запрос к GDELT
        project_id=env('GOOGLE_BIGQUERY_PROJECT_ID'),  # Указыываем свой ProjectID
        reauth=True
)
gbq_df.head(10)

,SourceCommonName,theme,count
0,yahoo.com,,1019666
1,entornointeligente.com,,929940
2,lopinion.ma,,685675
3,krankenkassen-direkt.de,,551574
4,gaeubote.de,,512467
5,surenio.com.ar,,496382
6,skyscrapercity.com,,457249
7,fasopresse.net,,442445
8,ifeng.com,,387729
9,sina.com.tw,,379083


In [26]:
import gdelt

# Version 2 queries
gd2 = gdelt.gdelt(version=2)

# Single 15 minute interval pull, output to json format with mentions table
results = gd2.Search('2016 Nov 1', table='mentions',output='json')
print(len(results))

# Full day pull, output to pandas dataframe, events table
results = gd2.Search(['2016 11 01'],table='events',coverage=True)
print(len(results))

3915757


/opt/tljh/user/lib/python3.7/site-packages/gdelt/parallel.py:67: UserWarning: GDELT does not have a url for date time 20161101233000
  warnings.warn(message)
/opt/tljh/user/lib/python3.7/site-packages/gdelt/parallel.py:103: UserWarning: GDELT did not return data for date time 20161101233000
  warnings.warn(message)


226375


In [27]:
results

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,594558341,20151102,201511,2015,2015.8274,NaN,NaN,NaN,NaN,NaN,...,4,"Thabeban, Queensland, Australia",AS,AS04,154665,-24.9000,152.3500,-1604217,20161101010000,http://www.news-mail.com.au/news/locals-aim-to...
1,594558342,20151102,201511,2015,2015.8274,NaN,NaN,NaN,NaN,NaN,...,4,"Balaka, , Central African Republic",CT,CT00,18585,3.8000,17.6500,-1339037,20161101010000,http://mainichi.jp/english/articles/20161101/p...
2,594558343,20151102,201511,2015,2015.8274,NaN,NaN,NaN,NaN,NaN,...,4,"Puttur, Andhra Pradesh, India",IN,IN02,70251,13.4500,79.5500,-2108469,20161101010000,http://www.thehindu.com/news/cities/Mangalore/...
3,594558344,20151102,201511,2015,2015.8274,NaN,NaN,NaN,NaN,NaN,...,4,"Puttur, Andhra Pradesh, India",IN,IN02,70251,13.4500,79.5500,-2108469,20161101010000,http://www.thehindu.com/news/cities/Mangalore/...
4,594558345,20151102,201511,2015,2015.8274,NaN,NaN,NaN,NaN,NaN,...,3,"Des Moines, Iowa, United States",US,USIA,NaN,41.6005,-93.6091,465961,20161101010000,http://whoradio.iheart.com/articles/who-radio-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226370,594865260,20161101,201611,2016,2016.8247,pan,PUNJABI,NaN,NaN,pan,...,4,"Shah Hussain, Sindh, Pakistan",PK,PK05,23687,27.4722,68.4708,6131793,20161101223000,http://www.tribuneindia.com/news/comment/when-...
226371,594865261,20161101,201611,2016,2016.8247,panGOV,PUNJABI,NaN,NaN,pan,...,4,"Sultan Bahu, Punjab, Pakistan",PK,PK04,23661,30.8024,71.8745,6130091,20161101223000,http://www.tribuneindia.com/news/comment/when-...
226372,594865262,20161101,201611,2016,2016.8247,urd,URDU,NaN,NaN,urd,...,1,Greece,GR,GR,NaN,39.0000,22.0000,GR,20161101223000,http://www.citylab.com/design/2016/11/language...
226373,594865263,20161101,201611,2016,2016.8247,urd,URDU,NaN,NaN,urd,...,1,Greece,GR,GR,NaN,39.0000,22.0000,GR,20161101223000,http://www.citylab.com/design/2016/11/language...


In [3]:
from gydelt.gydelt import GetData, ProcessData 

GD = GetData()
# data = GD.read_from_file(path='sample data/fromGKG.txt', parse_dates=['Date'])
data = GD.fire_query(
    project_id=env('GOOGLE_BIGQUERY_PROJECT_ID'), 
    search_dict={
        'Locations': 'China;;', 
        'Persons': 'Donald Trump;;'
    }, auth_file='./service_account.json',
    limit=1000
)
# This would mean that the 'Locations' should have BOTH 'United States' and 'China' and 'Persons' should NOT have 'Donald Trump'
data.head(5)


This query will process 861.89 GB.
Proceed ?(Y/N)
Y


Downloading: 100%|██████████| 1000/1000 [00:00<00:00, 1230.24rows/s]

Success: Data frame saved as - /storage/home/jupyter-valya/COMPTECH-1\Result(2021-01-25 07.48.32).csv


,Date,Themes,Locations,Persons,Organizations,ToneData
0,20200302000000,LGBT;TAX_FNCACT;TAX_FNCACT_CANDIDATE;USPEC_POL...,"1#Afghanistan#AF#AF#33#66#AF;2#New Hampshire, ...",bernie sander,None,"2.56410256410256,5.76923076923077,3.2051282051..."
1,20200302000000,TAX_ETHNICITY;TAX_ETHNICITY_AUSTRALIAN;CRISISL...,1#China#CH#CH#35#105#CH;1#Australia#AS#AS#-25#...,julia lee;julia lee julialee,None,"-4.47227191413238,1.43112701252236,5.903398926..."
2,20200302000000,UNGP_FORESTS_RIVERS_OCEANS;ECON_MOU;WB_135_TRA...,"4#Braemar, New South Wales, Australia#AS#AS02#...",havilah maldorky;chris giles,None,"1.94552529182879,1.94552529182879,0,1.94552529..."
3,20200302000000,EPU_POLICY;EPU_POLICY_POLITICAL;POLITICAL_TURM...,1#Iraq#IZ#IZ#33#44#IZ,mohammed tawfiq allawi,None,"-9.41176470588235,1.17647058823529,10.58823529..."
4,20200302000000,DEMOCRACY;LEADER;TAX_FNCACT;TAX_FNCACT_PRESIDE...,"3#South Bend, Indiana, United States#US#USIN#4...",pete buttigieg,None,"0.709219858156029,2.83687943262411,2.127659574..."


In [4]:
PD = ProcessData(data_frame=data)

In [5]:
# Calling the wrapper function to pre-process the whole data
processed_data_1 = PD.pre_process()
processed_data_1.head(5)


Time taken for pre-processing the data --> 0.13 seconds


,Date,Themes,Locations,Persons,Organizations,ToneData,Countries,Tone,Positive Score,Negative Score,Polarity,Activity Reference Density,Self/Group Reference Density,Word Count
0,20200302000000,LGBT;TAX_FNCACT;TAX_FNCACT_CANDIDATE;USPEC_POL...,"1#Afghanistan#AF#AF#33#66#AF;2#New Hampshire, ...",bernie sander,unknown,"2.56410256410256,5.76923076923077,3.2051282051...",Afghanistan;United States,2.564103,5.769231,3.205128,8.974359,21.794872,0.000000,146.0
1,20200302000000,TAX_ETHNICITY;TAX_ETHNICITY_AUSTRALIAN;CRISISL...,1#China#CH#CH#35#105#CH;1#Australia#AS#AS#-25#...,julia lee;julia lee julialee,unknown,"-4.47227191413238,1.43112701252236,5.903398926...",China;Australia,-4.472272,1.431127,5.903399,7.334526,20.751342,1.252236,490.0
2,20200302000000,UNGP_FORESTS_RIVERS_OCEANS;ECON_MOU;WB_135_TRA...,"4#Braemar, New South Wales, Australia#AS#AS02#...",havilah maldorky;chris giles,unknown,"1.94552529182879,1.94552529182879,0,1.94552529...",Australia,1.945525,1.945525,0.000000,1.945525,26.070039,1.167315,246.0
3,20200302000000,EPU_POLICY;EPU_POLICY_POLITICAL;POLITICAL_TURM...,1#Iraq#IZ#IZ#33#44#IZ,mohammed tawfiq allawi,unknown,"-9.41176470588235,1.17647058823529,10.58823529...",Iraq,-9.411765,1.176471,10.588235,11.764706,20.000000,0.000000,80.0
4,20200302000000,DEMOCRACY;LEADER;TAX_FNCACT;TAX_FNCACT_PRESIDE...,"3#South Bend, Indiana, United States#US#USIN#4...",pete buttigieg,unknown,"0.709219858156029,2.83687943262411,2.127659574...",United States;India,0.709220,2.836879,2.127660,4.964539,19.858156,0.000000,134.0
